# Global Imports

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
rcParams['axes.color_cycle']

['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628']

### External Package Imports

In [ ]:
from __future__ import division

In [2]:
import os as os
import pickle as pickle
import pandas as pd

### Module Imports

In [7]:
from Helpers.LinAlg import *
from Helpers.Pandas import *
from Helpers.RPY2 import *
from Stats.Scipy import *
from Stats.Regression import *

from Figures.FigureHelpers import *
from Figures.Pandas import *
from Figures.Boxplots import *
from Figures.Regression import *

### Tweaking Display Parameters

In [5]:
pd.set_option('precision', 3)
pd.set_option('display.width', 300)
plt.rcParams['font.size'] = 14

In [ ]:
import seaborn as sns

sns.set_context("paper", font_scale=1.7, rc={"lines.linewidth": 2.5})
sns.set_style("white")

In [6]:
'''Color schemes for paper taken from http://colorbrewer2.org/'''
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', 
          '#ffff33', '#a65628']
colors_st = ['#CA0020', '#F4A582', '#92C5DE', '#0571B0']
colors_th = ['#E66101', '#FDB863', '#B2ABD2', '#5E3C99']

Preform a logit transformation while constraining extreme values.

In [1]:
from scipy.special import logit

logit_adj = lambda df: logit(df.clip(.001, .999))

### Model Fit/Assessment Helpers

In [1]:
def detrend(x,y):
    x, y = match_series(x, y)
    reg = linear_regression(x, y)
    adj = (y - reg['intercept']) / reg['slope']
    return adj

In [3]:
def get_error(x, y, groups=None, denominator='mean'):
    if groups is not None:
        r = pd.DataFrame({g: get_error(x.ix[ti(groups==g)], y, denominator=denominator) 
                          for g in groups.unique()})
        return r
    if denominator == 'x':
        d = x
    elif denominator == 'y':
        d = y
    else:
        d = (x + y) / 2.
    
    pct_error = ((x - y) / d).abs().dropna().mean() * 100.
    avg_error = (x- y).abs().dropna().mean()
    return pd.Series({'error (years)': avg_error, '% error': pct_error})

In [ ]:
def model_fit(m, age):
    reg = linear_regression(age, m)
    error = get_error(age, m)
    vec = pd.Series({'n': int(len(m.dropna())), 
                     'r': reg['r-value'],
                     'error (years)': error['error (years)'],
                     '% error': error['% error']})
    return vec.ix[['n','r','error (years)', '% error']]

Preform two step linear adjustement. 
* First on each data-source seperately
* Then on all data-sources combined

In [4]:
def two_step_adjustment(m1, m2, age, groups):
    m1_adj = {}
    m2_adj = {}
    k = m1.index.intersection(m2.index)
    for s in groups.unique():
        pts = ti(groups == s).intersection(k)
        m1_adj[s] = detrend(age, m1.ix[pts])
        m2_adj[s] = detrend(age, m2.ix[pts])
    m1_adj = pd.concat(m1_adj.values())
    m2_adj = pd.concat(m2_adj.values())
    mc_adj = (m1_adj + m2_adj) / 2.
    mc_adj = detrend(age, mc_adj)
    mc_adj.name = 'Predicted Age (Combined)'
    return m1_adj, m2_adj, mc_adj

Manhattan plot

In [3]:
def manhattan(vec, chrom, coords, ax=None, ybound=None,
              flow='up', ticks=True):
    fig, ax = init_ax(ax, figsize=(9,3))
    x = 0
    chr_coords = []
    for i,c in enumerate(map(str, range(1,23))):
        v = vec.ix[ti(chrom == c)].dropna()
        series_scatter(coords + x, v, s=10, ax=ax, 
                       color=colors[i % 5], 
                       ann=None, alpha=1) 
        chr_len = coords.ix[v.index].max()
        x = x + chr_len + 3e7
        chr_coords += [x - (chr_len / 2.)]
    ax.set_xbound(-3e7, x + 3e7)
    if ybound is not None:
        ax.set_ybound(ybound[0],ybound[1])
    ax.set_xlabel('Chromosome')
    
    if ticks:
        ax.set_xticks(chr_coords)
        ax.set_xticklabels(map(str, range(1,23)))
    else:
        ax.set_xticks([])
    top = flow == 'down'
    prettify_ax(ax, top)

### Global Parameters

While it is not always the best practice to set global parameters, I find that for analyses such as these it is beneficial for a few reasons.  
* Selection criteria can be established in one place and updates can be propagated to all subsequent analyses  
* Things such as hard paths can be extracted from the logic and placed in one central location

In [27]:
FIGDIR = '/cellar/users/agross/figures/'

path = '/cellar/users/agross/TCGA_Code/Methlation/'
ucsd_path = path + 'data/UCSD_Methylation/'
hannum_path = path + 'data/Hannum/'
path_italy = path + 'data/EPIC_ITALY/'